1/11 改进字段 
----
患者入院前12小时前 *  
当日 当天  *  
2018-05-25 (2018-05-25 ) *  
颈2-4  颈2018-02-04  *   
2018-05-22 (2018-05-22 )  *  
患者2018-05-19 （2018-05-19 ）晚上2018-05-19 20:00  *    
曾于当日上午至我院急诊就诊， *   
患者入院前1天（2018-05-20 18：05）在活  患者2018-05-19 （2018-05-20 18:05）  *  
患者入院前3天12点左右，  患者2018-05-16 12点左右， *   
2018-05-08 (2018-05-09 )  *    
L2018-03-04 椎间盘膨隆，L2018-04-05 、L5-S1椎间盘突出伴变性。  *    
患者入院当天早晨8：00左右，  患者2018-05-15早晨2018-05-15 8:00左右， *   


#### 输入：  
text:【现病史】段落文字 string  
recorddate: 记录时间 %Y-%m-%d string  
entrydate: 入院时间 %Y-%m-%d %hh:mm string  
#### 输出：
text string  

In [1]:
text = """2018-01-01 当天
患者于入院3天前(2018-05-25)
患者入院当天早晨8：00左右，
患者入院前1天（2018-05-20 18：05），入院前2周（2018-05-01）
颈2-4,L3-4椎间盘膨隆
入院当天，
患者入院前12小时前，
患者2018.1.16 7点左右，
入院前3日，
于15日下午5点
当日 当天
患者入院前15天出现头晕不适，伴视物旋转，无肢体活动受限制经休息后头晕症状可稍缓解，
近2日患者头晕症状较前加重，伴恶心，无呕吐，无头痛，无耳鸣、听力下降，无呕血、黑便，无视物模糊，
无四肢抽搐、短暂意识模糊或意识丧失，无进食反呛，无言语含糊、面部及肢体麻木乏力感，无胸闷、心悸、
活动后气急、浮肿无视物左右重影。遂来院就诊，4-13查头颅CT平扫提示：双侧基底节及半卵圆区散在缺血灶。
"""

In [2]:
import re
from datetime import datetime, timedelta

def standardize_time(text,recorddate,entrydate):#main function
    
    recorddate = recorddate
    entrydate = entrydate
    
    #规范时间分隔符
    text = re.sub('－','-',text)
    text = re.sub('：',':',text)
    text = re.sub(r"(\d{4})\.(\d{1,2})\.(\d{1,2})",r"\1-\2-\3",text)
    
    #替换(入院)X天前等
    text = re.sub(r"((入院)?((前([0-9半零一二两三四五六七八九十]+)([小时天日周月年]+))|(([0-9半零一二两三四五六七八九十]+)([小时天日周月年]+)前)))|(入院当[天日])",
                  lambda m: entry_day_sub(m,recorddate,entrydate),
                  text)
    
    #修正替换后时间重复的字段 
    #入院前2周（2018-05-01）--> 2018-05-25 (2018-05-25 )  默认保留括号里的datetime
    text = re.sub(r"\d{4}-\d{2}-\d{2}\s*[(（](\d{4}-\d{2}-\d{2}\s*(\d{2}:\d{2})?)[)）]",
                  r"\1",
                  text)
    
    #替换中文日期
    chinese_pattern = re.compile(r"(((\d{4}[年-])?(1[0-2]|0?[1-9])[月-](3[01]|[12][0-9]|0?[1-9])日?)|(于(3[01]|[12][0-9]|0?[1-9])日))\s*，?([上中下午晚夜间早凌清晨起]+)?((2[0-4]|1[0-9]|0?[1-9])[点:：时]([0-5]?[0-9]分?)?)?")
    text = re.sub(chinese_pattern, 
                  lambda m: chinesedate(m,recorddate),
                  text)
    
    #替换昨天今天当天
    text = re.sub(r"([今昨][天日]?|当[日天])([上中下午晚夜间早凌清晨起]+)?((2[0-4]|1[0-9]|0?[1-9])[点:：时]([0-5]?[0-9]分?)?)?",
                  lambda m: today_yesterday(m,recorddate,text),
                  text)
    
    #只有时间的补上日期
    text = add_date2time(text,recorddate)
    
    #修正匹配过度的字段
    #颈2-4 L3-4椎间盘膨隆，L4-5、L5-S1椎间盘突出伴变性
    text = re.sub(r"颈\d{4}-(\d{2}-\d{2})",lambda m: '颈'+m.group(1)[1]+'-'+m.group(1)[4] ,text)
    text = re.sub(r"L\d{4}-(\d{2}-\d{2})",lambda m: 'L'+m.group(1)[1]+'-'+m.group(1)[4] ,text)

    return text


def chinesedate(match,recorddate):#中文日期替换
    if match.group() is None:
        return None
    
    else:
        print('匹配1:', match.group())
        res = {
            "year":match.group(3),
            "month": match.group(4),
            "day": match.group(5) if match.group(5) is not None else match.group(7),
            "hour": match.group(10) if match.group(10) is not None else '00',
            "minute": match.group(11) if match.group(11) is not None else '00'
        }
        params = {}

        for name in res:
            tmp = None
            if res[name] is not None:
                tmp = re.search(r"[0-9]+",res[name]).group()
            if tmp is not None:
                params[name] = int(tmp)
        target_date = datetime.strptime(recorddate+' 00:00','%Y-%m-%d %H:%M').replace(**params) ##默认为记录时间
        is_pm = match.group(8)
        
        if match.group(9) is None:#没有时间 只有日期
            if is_pm is None: 
                is_pm = ""
            result = str(target_date).split()[0]+' '+is_pm

        else: 
            if is_pm is not None and len(is_pm) != 0:
                if re.match(r"[下午晚夜间]+",is_pm) is not None:
                    hour = target_date.time().hour
                    if hour < 12:
                        target_date = target_date.replace(hour=hour + 12)
            result = str(target_date.strftime('%Y-%m-%d %H:%M'))
        
        print('改为:',result)
        return result


def cn2dig(src): #中文数字
    UTIL_CN_NUM = {
    '零': 0,'半':0.5, '一': 1, '二': 2, '两': 2, '三': 3, '四': 4,
    '五': 5, '六': 6, '七': 7, '八': 8, '九': 9,'十':10
    }
    if src == "":
        return None
    m = re.match("\d+", src)
    if m:
        return int(m.group())
    else:
        if src in UTIL_CN_NUM.keys():
            return int(UTIL_CN_NUM[src])
        else:
            return None

def entry_day_sub(match,recorddate,entrydate):
    numdays = {'日':1,'天':1,'周':7,'月':30,'年':365}
    if match.group() is None:
        return None
    else:
        print('匹配2:', match.group())
        if match.group(2) is not None: #入院？
            date = datetime.strptime(entrydate,'%Y-%m-%d %H:%M') #transfer to datetime
        else: 
            date = datetime.strptime(recorddate,'%Y-%m-%d')
        
        if match.group(10) is not None:#入院当天
            target = str(entrydate).split()[0]
        
        if match.group(3) is not None:#前...or...前
            params = {
                "digit": cn2dig(match.group(5)) if match.group(5) is not None else cn2dig(match.group(8)),
                "timetype": match.group(6) if match.group(6) is not None else match.group(9)
            }
            if params["timetype"] == '小时': #小时
                target = str((date-timedelta(hours=params["digit"])).strftime('%Y-%m-%d %H:%M'))
            else: #年月周日
                target = str(date-timedelta(days=params["digit"]*numdays[params["timetype"]])).split()[0]+' '
            
        print('改为:', target)
        return target

def today_yesterday(match,recorddate,text): #今当昨天
    keyDate = {'昨天':-1,'昨日':-1,'昨':-1,'今天': 0,'今日': 0, '今':0}
    is_pm = match.group(2)
    
    if match.group() is None:
        return None
    else:
        print('匹配3:',match.group())
        
        #处理“当天” -->最近日期
        if match.group(1) == '当天' or match.group(1) == '当日':
            dates = re.findall(r"(\d{4}-\d{1,2}-\d{1,2})|(当[天日])",text)
            tmp = None
            for d in dates:
                if d[0] != "": tmp = d[0] #储存最近一次出现的日期
                if d[0] == "" and d[1] != "":
                    break
            if tmp is None:
                closestdate = recorddate
            else:
                closestdate = tmp
            target = datetime.strptime(closestdate,'%Y-%m-%d') 

        #处理当天 --> 段首日期                 
#             firstdate = re.search(r"(\d{4}-\d{1,2}-\d{1,2})",text)
#             if firstdate is None:#若段前没有日期则默认为记录时间
#                 firstdate = recorddate
#             else: 
#                 firstdate = firstdate.group()
#             target = datetime.strptime(firstdate,'%Y-%m-%d')
        
            
        else:#处理“今昨”
            target = datetime.strptime(recorddate,'%Y-%m-%d') + timedelta(days=keyDate[match.group(1)])
        
        if match.group(3) is None:#如果只有日期 return date only
            if match.group(2) is not None: #保留下午上午类的词
                result = str(target).split()[0]+ match.group(2)
            else:
                result = str(target).split()[0]+' '
            
        else:#若有日期+时间
            res = {
                "hour": match.group(4) if match.group(4) is not None else '00',
                "minute": match.group(5) if match.group(5) is not None else '00'
            }
            params = {}

            for name in res:
                tmp = int(re.search(r"[0-9]+",res[name]).group())
                params[name] = tmp
                if name == 'hour' and tmp == 24: params[name] = 0 #if hour == 24: hour = 0
            
            target = target.replace(**params)
            if is_pm is not None:
                if re.match(r"[下午晚夜间]+",is_pm) is not None:
                    hour = target.time().hour 
                    if hour < 12:
                        target = target.replace(hour=hour + 12)
            result = str(target.strftime('%Y-%m-%d %H:%M'))
        print('改为:', result)
        return result

def added_date(sample,record_date): 
    m = re.findall(r"(\d{4}-\d{2}-\d{2})?\s?(\d{1,2}[:：]\d{1,2})?",sample) 
    tmp = None
    lst = {}
    for match in m:
        if match[0] != "": tmp = match[0] #储存最近一次出现的日期
        if match[0] == "" and match[1] != "":
            if tmp is not None: 
                time, added = match[1], tmp+' '+match[1].replace('：',':') 
            else:
                time, added = match[1], str(record_date)+' '+match[1].replace('：',':')
            lst[time] = added
    return lst

def add_date2time(sample, recorddate): #补日期
    lst = added_date(sample, recorddate)
    tmp = sample
    for date in lst:
        print('匹配4:', date)
        print('改为:', lst[date])
        tmp = re.sub(date,lst[date],tmp)
    return tmp


显示匹配更改结果:

In [3]:
new = standardize_time(text,'2018-04-16','2018-04-16 3:00')

匹配2: 入院3天前
改为: 2018-04-13 
匹配2: 入院当天
改为: 2018-04-16
匹配2: 入院前1天
改为: 2018-04-15 
匹配2: 入院前2周
改为: 2018-04-02 
匹配2: 入院当天
改为: 2018-04-16
匹配2: 入院前12小时
改为: 2018-04-15 15:00
匹配2: 入院前3日
改为: 2018-04-13 
匹配2: 入院前15天
改为: 2018-04-01 
匹配1: 2018-01-01 
改为: 2018-01-01 
匹配1: 2018-05-25

改为: 2018-05-25 
匹配1: 2018-04-16早晨8:00
改为: 2018-04-16 08:00
匹配1: 2018-05-20 18:05
改为: 2018-05-20 18:05
匹配1: 2018-05-01

改为: 2018-05-01 
匹配1: 2-4
改为: 2018-02-04 
匹配1: 3-4
改为: 2018-03-04 
匹配1: 2018-04-16，
改为: 2018-04-16 
匹配1: 2018-04-15 15:00
改为: 2018-04-15 15:00
匹配1: 2018-1-16 7点
改为: 2018-01-16 07:00
匹配1: 2018-04-13 ，
改为: 2018-04-13 
匹配1: 于15日下午5点
改为: 2018-04-15 17:00
匹配1: 2018-04-01 
改为: 2018-04-01 
匹配1: 4-13
改为: 2018-04-13 
匹配3: 当天
改为: 2018-01-01 
匹配3: 当日
改为: 2018-01-01 
匹配3: 当天
改为: 2018-01-01 


In [4]:
#更改完的段落
print(new)

2018-01-01 2018-01-01 
患者于2018-05-25 患者2018-04-16 08:00左右，
患者2018-05-20 18:05，2018-05-01 颈2-4 ,L3-4 椎间盘膨隆
2018-04-16 
患者2018-04-15 15:00前，
患者2018-01-16 07:00左右，
2018-04-13 
2018-04-15 17:00
2018-01-01  2018-01-01 
患者2018-04-01 出现头晕不适，伴视物旋转，无肢体活动受限制经休息后头晕症状可稍缓解，
近2日患者头晕症状较前加重，伴恶心，无呕吐，无头痛，无耳鸣、听力下降，无呕血、黑便，无视物模糊，
无四肢抽搐、短暂意识模糊或意识丧失，无进食反呛，无言语含糊、面部及肢体麻木乏力感，无胸闷、心悸、
活动后气急、浮肿无视物左右重影。遂来院就诊，2018-04-13 查头颅CT平扫提示:双侧基底节及半卵圆区散在缺血灶。
胸部CT未见异常。查心电图示:窦性心律，T波改变。查肝肾功能、电解质、心肌酶、血淀粉酶、Tnl、BNP、凝血功能、
D-2聚体、血常规、CRP基本正常。给予“七叶皂苷钠、长春西汀，甘露醇，依达拉奉”等治疗后，头晕较前有好转，
为进一步诊治，拟"头晕"收住入病房。
    患者自发病以来，神清，精神可，胃纳、夜眠可，二便如常，否认近期体重减轻。既往疾病正在药物治疗



In [5]:
def record_date(sample):#提取记录时间和入院时间
    record = re.findall(r"记录时间：(\d{4}-\d{2}-\d{2})", sample)
    entry = re.findall(r"入院时间：(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2})", sample)
    return ''.join(record), ''.join(entry)

filelst = []
with open('filename.csv') as p:
    for line in p:
        if 'File' in line:
            filelst.append(line)
            
filenames = [file.replace('/','\\').split('\\') for file in filelst]
names = [e for line in filenames for e in line if 'txt' in e]


folder = '神经内科17-12至18-06病历/File/'

with open('时间替换.txt','w') as newfile:
    for file in names:
        path = folder+file
        print('文件名:',file)
        with open(path,encoding="gbk") as f:
            sample = f.read()

        recorddate = record_date(sample)[0]
        entrydate = record_date(sample)[1]
        text = sample.split('【')[2]
        
        newfile.write('文件名:'+str(file)+'\n')
        newfile.write('记录时间'+recorddate+'\n')
        newfile.write('入院时间'+entrydate+'\n')
        newfile.write(text)
        newfile.write('--------------------------------------\n')
        newfile.write(standardize_time(text,recorddate,entrydate))
        newfile.write('=============================================================\n\n')

文件名: af8e4ba2-7c9b-43a3-bb30-be24a3b466de.txt
匹配2: 1月前
改为: 2018-04-18 
匹配1: 2018-05-17  晚上20:30
改为: 2018-05-17 20:30
匹配1: 5-18
改为: 2018-05-18 
匹配1: 5-18 
改为: 2018-05-18 
匹配1: 2018-04-18 
改为: 2018-04-18 
文件名: 620ae858-156e-4e66-b865-d358168233e2.txt
匹配2: 入院前两天
改为: 2018-05-19 
匹配1: 2018-05-19 
改为: 2018-05-19 
文件名: bfe7426e-2d2b-458b-ae12-d11a4b69e47f.txt
匹配1: 5月24日夜间22:00
改为: 2018-05-24 22:00
匹配4: 23:40
改为: 2018-05-24 23:40
文件名: 57c258df-ef3e-4f7e-88b9-53aef1190b17.txt
匹配1: 5月25日
改为: 2018-05-25 
文件名: d3fcf56f-b2a6-4f6a-95eb-032a911f6f16.txt
匹配2: 入院前两周
改为: 2018-05-13 
匹配2: 入院当天
改为: 2018-05-27
匹配1: 2018-05-13 
改为: 2018-05-13 
匹配1: 2018-05-27
改为: 2018-05-27 
文件名: 74c90838-d7d6-479a-963e-052254b15632.txt
匹配2: 入院当天
改为: 2018-05-27
匹配1: 2018-05-27晨起
改为: 2018-05-27 晨起
文件名: 49ee580c-51fc-4d97-a034-fe614540f455.txt
匹配1: 5月25日
改为: 2018-05-25 
匹配1: 5月26日
改为: 2018-05-26 
文件名: 16a58c37-4725-45cc-91b8-f415e8e6d8ac.txt
匹配1: 5月27日15:00
改为: 2018-05-27 15:00
匹配3: 当日15:45
改为: 2018-05-27 15:45
匹配4: 15:47
改为:

匹配2: 入院4天前
改为: 2018-05-10 
匹配1: 2018-05-10 ，
改为: 2018-05-10 
匹配1: 2018-05-12
改为: 2018-05-12 
文件名: 07915a61-441f-4ebf-9fda-9099de86b289.txt
匹配2: 10年前
改为: 2008-05-09 
匹配1: 2018-05-06  9:30
改为: 2018-05-06 09:30
匹配1: 5-6 11:17
改为: 2018-05-06 11:17
匹配1: 2008-05-09 
改为: 2008-05-09 
匹配4: 11:32
改为: 2018-05-06 11:32
文件名: 83c90e86-5aa0-4ca2-b3c1-28849481cd51.txt
匹配1: 2018-05-10晨起8点
改为: 2018-05-10 08:00
匹配4: 13:45
改为: 2018-05-10 13:45
匹配4: 14:19
改为: 2018-05-10 14:19
匹配4: 14:27
改为: 2018-05-10 14:27
文件名: 9d1e9141-366b-4df9-a43b-bbd77ba0478d.txt
匹配2: 入院3天前
改为: 2018-05-11 
匹配1: 2018-05-11 ，
改为: 2018-05-11 
匹配1: 2018-05-12 
改为: 2018-05-12 
文件名: 946a48e0-5571-4847-9055-7eff0ab5a414.txt
匹配1: 2018-5-11
改为: 2018-05-11 
文件名: 29135db4-bee5-4ad7-a099-626a9720d2e0.txt
匹配1: 2018-05-15
改为: 2018-05-15 
匹配3: 昨日
改为: 2018-05-14 
匹配3: 今晨
改为: 2018-05-15晨
文件名: df5cdc11-9035-4e85-9242-aa4c1b630aaa.txt
匹配3: 今凌晨3点
改为: 2018-05-16 03:00
匹配4: 5:32
改为: 2018-05-16 5:32
匹配4: 5:59
改为: 2018-05-16 5:59
文件名: 5a538605-33f1-4015-b9a

匹配2: 入院4小时前
改为: 2018-04-30 10:55
匹配1: 2018-04-30 10:55
改为: 2018-04-30 10:55
匹配1: 4-29 16:30
改为: 2018-04-29 16:30
匹配1: 4-29 21:12
改为: 2018-04-29 21:12
文件名: 0b77d51b-b005-4871-a188-eb65c459ff9d.txt
匹配1: 2018-04-28晨起6点
改为: 2018-04-28 06:00
匹配1: 2018-04-29 09:03
改为: 2018-04-29 09:03
匹配1: 2018-05-01 14:02
改为: 2018-05-01 14:02
匹配4: 11:12
改为: 2018-04-29 11:12
文件名: 9e44c246-8d81-4b38-a388-7da50ba9de02.txt
匹配1: 2018-05-01下午5点
改为: 2018-05-01 17:00
匹配3: 今日10:00
改为: 2018-05-02 10:00
匹配4: 11:17
改为: 2018-05-02 11:17
匹配4: 11:30
改为: 2018-05-02 11:30
文件名: 33ec88e4-f320-432e-9019-84a66cbf4216.txt
匹配1: 2018-05-2晨起
改为: 2018-05-02 晨起
匹配3: 今日下午
改为: 2018-05-02下午
匹配4: 14:00
改为: 2018-05-02 14:00
文件名: 1aa6bb8f-dc9b-4498-9be1-0fe6651d8066.txt
匹配1: 2018-05-01上午7点
改为: 2018-05-01 07:00
匹配1: 5-2
改为: 2018-05-02 
匹配1: 5-2 19:41
改为: 2018-05-02 19:41
文件名: 4fb408bc-a814-4405-a4ae-7148c7750c25.txt
匹配2: 入院前2天
改为: 2018-04-25 
匹配1: 2018-04-25 
改为: 2018-04-25 
文件名: 62cbf593-128c-4f3a-8b7d-d2f261230e03.txt
匹配2: 入院2天前
改为: 2018-

匹配2: 入院前一天
改为: 2018-04-19 
匹配1: 2018-04-19 晨起
改为: 2018-04-19 晨起
文件名: 2081d278-4e9e-4be4-aaa2-f1d954535a9e.txt
匹配2: 入院前一天
改为: 2018-04-22 
匹配1: 2018-04-22 凌晨
改为: 2018-04-22 凌晨
文件名: 2a2ab6c6-e3e7-47bb-9a7f-5db91f63e2d9.txt
匹配1: 4月23日下午16:30
改为: 2018-04-23 16:30
匹配3: 当日19:20
改为: 2018-04-23 19:20
匹配4: 19:23
改为: 2018-04-23 19:23
匹配4: 19:44
改为: 2018-04-23 19:44
匹配4: 19:50
改为: 2018-04-23 19:50
文件名: 4c438fa2-2950-415f-8fcf-cd7d1388647a.txt
匹配1: 2018-04-18下午15点
改为: 2018-04-18 15:00
匹配4: 15:50
改为: 2018-04-18 15:50
匹配4: 16:38
改为: 2018-04-18 16:38
文件名: b2cb0ac6-4414-4709-966c-bc5a55184daf.txt
匹配2: 入院前1天
改为: 2018-04-18 
匹配1: 2018-04-18下午17:30
改为: 2018-04-18 17:30
匹配4: 18:53
改为: 2018-04-18 18:53
匹配4: 19:14
改为: 2018-04-18 19:14
文件名: 15f5be0b-3fa8-4b21-a6b4-36b7cfafc761.txt
匹配1: 2018-04-20晨起2点08分
改为: 2018-04-20 02:08
匹配4: 03:08
改为: 2018-04-20 03:08
匹配4: 03:21
改为: 2018-04-20 03:21
匹配4: 03:51
改为: 2018-04-20 03:51
匹配4: 0:20
改为: 2018-04-20 0:20
匹配4: 4:40
改为: 2018-04-20 4:40
匹配4: 4:49
改为: 2018-04-20 4:49
文件

匹配2: 入院前3天
改为: 2018-04-08 
匹配1: 2018-04-09上午7点30分
改为: 2018-04-09 07:30
文件名: d7036f4b-d12d-4cc2-ba29-3f090e072bf2.txt
匹配1: 2018-04-10 下午16点
改为: 2018-04-10 16:00
匹配1: 4-11
改为: 2018-04-11 
匹配4: 12:41
改为: 2018-04-11 12:41
文件名: 8d27045a-ff9e-4433-ad72-234a7bd5d1d3.txt
匹配2: 入院前15天
改为: 2018-03-29 
匹配1: 2018-03-29 
改为: 2018-03-29 
匹配1: 4-13
改为: 2018-04-13 
文件名: 000cdb73-c802-4774-88ae-b82d0fe1e16c.txt
匹配2: 入院前1周
改为: 2018-04-06 
匹配1: 2018-04-06 
改为: 2018-04-06 
匹配3: 今日12:52
改为: 2018-04-13 12:52
匹配4: 13:18
改为: 2018-04-13 13:18
匹配4: 13:10
改为: 2018-04-13 13:10
文件名: 708be36f-7a1b-4a29-907c-959ce7173e27.txt
匹配3: 昨下午14:30
改为: 2018-04-03 14:30
匹配4: 18:08
改为: 2018-04-03 18:08
匹配4: 18:25
改为: 2018-04-03 18:25
文件名: 1fd74b75-b4d3-4dce-a8c2-26b849c4c688.txt
匹配1: 2018-3-31 10点
改为: 2018-03-31 10:00
匹配4: 10:49
改为: 2018-03-31 10:49
文件名: 1a78a389-4161-462a-a60d-34ffb3bfd541.txt
匹配2: 入院前2天
改为: 2018-04-08 
匹配1: 2018-04-08 
改为: 2018-04-08 
匹配3: 当日
改为: 2018-04-08 
匹配3: 今日
改为: 2018-04-10 
文件名: ec3ecf84-0756-4862-a6ff

...

匹配2: 入院前20天
改为: 2017-12-25 
匹配1: 2017-12-25 ，
改为: 2017-12-25 
文件名: 2c47ea99-fb13-454a-95db-5c91c38b1c39.txt
匹配1: 2018-01-16晚上起
改为: 2018-01-16 晚上起
匹配1: 2018-01-17
改为: 2018-01-17 
匹配1: 2018-01-17  
改为: 2018-01-17 
匹配3: 昨晚起
改为: 2018-01-16晚起
匹配3: 今晨
改为: 2018-01-17晨
文件名: 72291419-f4e5-41c2-9d5e-ce000b151ee8.txt
匹配2: 入院前1天
改为: 2018-01-18 
匹配1: 2018-01-18 
改为: 2018-01-18 
匹配3: 今上午
改为: 2018-01-19上午
文件名: 59be48ac-d6a0-4344-b1b1-386a69e14472.txt
匹配2: 入院2天前
改为: 2018-01-07 
匹配1: 2018-01-07 
改为: 2018-01-07 
匹配1: 2017-05-17  1:00
改为: 2017-05-17 01:00
匹配1: 2018-01-08  
改为: 2018-01-08 
文件名: 9a8d593a-9a1f-4370-8fe6-16cb1b593b4b.txt
匹配1: 2018-01-17，下午4:00
改为: 2018-01-17 16:00
匹配4: 17:35
改为: 2018-01-17 17:35
匹配4: 11:06
改为: 2018-01-17 11:06
匹配4: 18:45
改为: 2018-01-17 18:45
文件名: c1c4e06a-ff4f-4546-8485-89aa86855e8f.txt
匹配2: 入院前4天
改为: 2018-01-08 
匹配1: 2018-01-08 ，
改为: 2018-01-08 
匹配1: 2018-1-11 13:35
改为: 2018-01-11 13:35
匹配4: 16:37
改为: 2018-01-11 16:37
文件名: 470f96e1-d51b-40f2-945b-00db46d788a0.txt
匹配1: 5-10


文件名: d1020c1a-d8ca-4b0b-8d26-0db77005d847.txt
匹配2: 入院一周前
改为: 2018-01-03 
匹配1: 2018-01-03 ，
改为: 2018-01-03 
匹配3: 今
改为: 2018-01-10 
文件名: c5c00129-c0be-407f-9c9d-e073a62def92.txt
匹配2: 入院前4天
改为: 2018-01-05 
匹配1: 2018-01-05 晨起起
改为: 2018-01-05 晨起起
匹配1: 2018-01-08 14:45
改为: 2018-01-08 14:45
匹配1: 2018-01-08
改为: 2018-01-08 
匹配1: 2018-01-08 
改为: 2018-01-08 
匹配1: 2018-01-08 
改为: 2018-01-08 
匹配1: 2018-01-09 
改为: 2018-01-09 
文件名: ad388c12-0554-414c-92b8-3116352e6d72.txt
匹配2: 入院1天前
改为: 2018-01-04 
匹配1: 2018-01-04 ，
改为: 2018-01-04 
匹配4: 09:00
改为: 2018-01-04 09:00
匹配4: 10:55
改为: 2018-01-04 10:55
匹配4: 11:24
改为: 2018-01-04 11:24
匹配4: 12:00
改为: 2018-01-04 12:00
文件名: a520ed5c-772b-4021-9176-8bea773b44da.txt
匹配1: 2018-1-1
改为: 2018-01-01 
匹配1: 2018-1-4
改为: 2018-01-04 
匹配1: 2018-1-7
改为: 2018-01-07 
文件名: 929353de-378a-439c-9a0e-456f9eba79c0.txt
匹配1: 2017-12-28起
改为: 2017-12-28 起
匹配1: 2017-12-31
改为: 2017-12-31 
匹配1: 2017-12-31
改为: 2017-12-31 
匹配1: 2018-01-03
改为: 2018-01-03 
匹配3: 当日
改为: 2017-12-31 
文件名: e5a1d8e7

匹配2: 12小时前
改为: 2017-12-18 12:00
匹配1: 2017-12-18 12:00
改为: 2017-12-18 12:00
匹配4: 21:50
改为: 2017-12-18 21:50
匹配4: 22:08
改为: 2017-12-18 22:08
文件名: 029548f4-f055-47dd-b657-cfa3bd582bc6.txt
匹配2: 入院前5天
改为: 2017-12-15 
匹配1: 2017-12-15 ，
改为: 2017-12-15 
文件名: 87acaaf3-eea0-4d6d-95f0-10c41f66de0a.txt
匹配2: 入院前6天
改为: 2017-12-16 
匹配1: 2017-12-16 
改为: 2017-12-16 
匹配1: 2-3
改为: 2017-02-03 
匹配1: 2-3
改为: 2017-02-03 
文件名: e4ae8513-8692-4911-8271-05dbba573996.txt
匹配2: 入院4天前
改为: 2017-12-18 
匹配1: 2017-12-18
改为: 2017-12-18 
匹配1: 2017-12-20  
改为: 2017-12-20 
匹配3: 今日
改为: 2017-12-22 
文件名: 9e0836ed-053e-4d79-9621-574df285c930.txt
匹配2: 3月前
改为: 2017-09-26 
匹配1: 2017-12-24，下午17:00
改为: 2017-12-24 17:00
匹配1: 12-25
改为: 2017-12-25 
匹配1: 2017-09-26 
改为: 2017-09-26 
文件名: 8e586f38-addc-434a-be03-493af638d7c1.txt
匹配1: 2017-12-14下午14:00
改为: 2017-12-14 14:00
匹配1: 12-7
改为: 2017-12-07 
匹配1: 12-13
改为: 2017-12-13 
匹配4: 14:30
改为: 2017-12-14 14:30
匹配4: 13:10
改为: 2017-12-14 13:10
匹配4: 15:53
改为: 2017-12-14 15:53
文件名: 22918828-bb12-4

匹配2: 2天前
改为: 2017-12-03 
匹配1: 2017-12-03 上午8:00
改为: 2017-12-03 08:00
匹配4: 9:15
改为: 2017-12-03 9:15
文件名: 26bb5c9e-0105-4852-ae9e-c8c4910d64da.txt
匹配2: 入院前1天
改为: 2017-12-10 
匹配1: 2017-12-09，晚上7:00
改为: 2017-12-09 19:00
匹配1: 2017-12-10 15:10
改为: 2017-12-10 15:10
匹配1: 5-1
改为: 2017-05-01 
匹配4: 15:38
改为: 2017-12-10 15:38
文件名: 8c9ee4e3-5755-4f89-a21a-8be3dab55faf.txt
匹配1: 2017-11-30晨起3:45
改为: 2017-11-30 03:45
匹配3: 昨晚21:00
改为: 2017-11-29 21:00
匹配4: 5:15
改为: 2017-11-29 5:15
匹配4: 5:32
改为: 2017-11-29 5:32
匹配4: 5:19
改为: 2017-11-29 5:19
文件名: 971d5259-4edf-4869-9297-af2db98661c7.txt
匹配1: 2017-11-30，下午15:00
改为: 2017-11-30 15:00
匹配4: 21:12
改为: 2017-11-30 21:12
匹配4: 22:31
改为: 2017-11-30 22:31
匹配4: 22:37
改为: 2017-11-30 22:37
文件名: e2939236-d665-4cef-8d83-caff1b96133d.txt
匹配1: 2017-12-04 22:00
改为: 2017-12-04 22:00
匹配3: 今日07:50
改为: 2017-12-05 07:50
匹配4: 8:03
改为: 2017-12-05 8:03
匹配4: 08:31
改为: 2017-12-05 08:31
文件名: b299652a-d62e-415d-828d-8288b07f18fd.txt
匹配2: 3月前
改为: 2017-09-07 
匹配1: 2017-12-5晨间
改为: 2017-12

文件名: fe6e3014-f6fe-45ea-9bee-1caab4c2cf83.txt
匹配1: 2017-11-15日晚上
改为: 2017-11-15 晚上
匹配1: 11月15日
改为: 2017-11-15 
文件名: e3e1c7b2-ea3b-47ea-b3d2-ab2fbfdee7e9.txt
匹配1: 2017-11-19 06:00
改为: 2017-11-19 06:00
匹配1: 11月21日10:00
改为: 2017-11-21 10:00
匹配4: 12:19
改为: 2017-11-21 12:19
匹配4: 13:12
改为: 2017-11-21 13:12
匹配4: 13:31
改为: 2017-11-21 13:31
文件名: e73790b6-8a9e-411c-b920-397e46ddc6a3.txt
匹配1: 11月18日18:30
改为: 2017-11-18 18:30
匹配3: 今日
改为: 2017-11-22 
文件名: 0c2de031-aab2-4b19-b455-138cad4ee05f.txt
匹配1: 2017-11-20 晚18点
改为: 2017-11-20 18:00
匹配4: 14:43
改为: 2017-11-20 14:43
匹配4: 15:41
改为: 2017-11-20 15:41
文件名: 3bd85239-b7a5-49f6-878c-d9984fb04989.txt
匹配1: 11月20日，
改为: 2017-11-20 
匹配1: 11月22日
改为: 2017-11-22 
文件名: 4d03b17c-5107-46a2-af55-8c3f94191d62.txt
匹配1: 2017-11-22下午14:00
改为: 2017-11-22 14:00
匹配4: 17:30
改为: 2017-11-22 17:30
匹配4: 17:14
改为: 2017-11-22 17:14
匹配4: 17:29
改为: 2017-11-22 17:29
匹配4: 17:35
改为: 2017-11-22 17:35
文件名: 70679aa6-2cb1-4280-9bc2-5a537fff9bda.txt
匹配1: 2017-11-10，上午9:00
改为: 2017-11-10 0